# Model Training

In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('./data/aad.csv')
df.head()

,Unnamed: 0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [24]:
df=df.drop(labels=['Unnamed: 0'],axis=1)

In [25]:
## Independent and dependent features
X = df.drop(labels=['Sales'],axis=1)
Y = df[['Sales']]

In [26]:
Y

,Sales
0,22.1
1,10.4
2,9.3
3,18.5
4,12.9
...,...
195,7.6
196,9.7
197,12.8
198,25.5


In [27]:
# Define which columns should be encode and which should be scaled
numerical_cols = X.select_dtypes(exclude='object').columns

In [28]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [29]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
    ])
preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols)
])

In [30]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [31]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [32]:
X_train.head()

,num_pipeline__TV,num_pipeline__Radio,num_pipeline__Newspaper
0,-1.598370,1.822785,2.158109
1,-1.599562,-1.313230,-1.310454
2,0.163763,-1.279726,-0.968285
3,-0.422424,-0.515825,-0.846416
4,1.796030,0.402197,-1.272956


In [33]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [34]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [35]:
regression.coef_

array([[3.72861919, 2.66697719, 0.10937166]])

In [36]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [37]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1.5958402221167631
MAE: 1.2654220059554535
R2 score 91.51734116027949


Lasso
Model Training Performance
RMSE: 2.4704192292805853
MAE: 1.9677297007641195
R2 score 79.67200688461735


Ridge
Model Training Performance
RMSE: 1.6082619107442553
MAE: 1.2773601644906003
R2 score 91.3847727109247


Elasticnet
Model Training Performance
RMSE: 3.0174610246573406
MAE: 2.454705887050813
R2 score 69.67250602795731




In [38]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']